In [ ]:
from python.analysis.NotebookUtils import init_notebook
%init_notebook

from rich import print
from python.analysis import Master, BeamParticleSelection, Plots, shower_merging, Tags, cross_section

cross_section.PlotStyler.SetPlotStyle(extend_colors = True, dpi = 140)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = "work/2023/cex_analysis_2GeV_confg.json"
if not config:
    mc = Master.Data("work/ROOTFiles/PDSPAnalyser/PDSPProd4a_MC_2GeV_reco1_sce_datadriven_v1_ntuple_v09_41_00_03.root", nTuple_type = Master.Ntuple_Type.PDSP)
    data = Master.Data("work/ROOTFiles/PDSPAnalyser/PDSPProd4_data_2GeV_reco2_ntuple_v09_42_03_01.root", nTuple_type = Master.Ntuple_Type.PDSP)

    beam_quality_fits = {
        "mc" : "work/2023/PDSPProd4a_MC_2GeV_reco1_sce_datadriven_v1_ntuple_v09_41_00_03/PDSPProd4a_MC_2GeV_reco1_sce_datadriven_v1_ntuple_v09_41_00_03_fit_values.json",
        "data" : "work/2023/PDSPProd4_data_2GeV_reco2_ntuple_v09_42_03_01/PDSPProd4_data_2GeV_reco2_ntuple_v09_42_03_01_fit_values.json"
    }
else:
    args = cross_section.ApplicationArguments.ResolveConfig(cross_section.LoadConfiguration(config))
    mc = Master.Data(args.mc_file, nTuple_type = Master.Ntuple_Type.PDSP)
    data = Master.Data(args.data_file, nTuple_type = Master.Ntuple_Type.PDSP)
    beam_quality_fits = {"mc" : args.mc_beam_quality_fit, "data" : args.data_beam_quality_fit}

samples = {"mc" : mc, "data": data}


for f in beam_quality_fits:
    beam_quality_fits[f] = cross_section.LoadSelectionFile(beam_quality_fits[f])

def FilterSamples(samples : dict, masks : dict, PFO : bool = False):
    for s in samples:
        if PFO:
            samples[s].Filter([masks[s]])
        else:
            samples[s].Filter([masks[s]], [masks[s]])

In [4]:
masks = {
    "mc" : BeamParticleSelection.PiBeamSelection(samples["mc"]),
    "data" : BeamParticleSelection.PiBeamSelection(samples["data"], use_beam_inst = True)
}
FilterSamples(samples, masks)
masks = {s : BeamParticleSelection.CaloSizeCut(samples[s]) for s in samples}
FilterSamples(samples, masks)
masks = {s : BeamParticleSelection.PandoraTagCut(samples[s]) for s in samples}
FilterSamples(samples, masks)
masks = {s : BeamParticleSelection.BeamQualityCut(samples[s], beam_quality_fits[s]) for s in samples}
FilterSamples(samples, masks)

number of entries before|after PiBeamSelection: 87403|87403
number of entries before|after PiBeamSelection: 232423|232423
number of entries before|after CaloSizeCut: 87403|87403
number of entries before|after CaloSizeCut: 232423|232423
number of entries before|after PandoraTagCut: 87403|87403
number of entries before|after PandoraTagCut: 232423|232423
number of entries before|after BeamQualityCut: 87403|59535
number of entries before|after BeamQualityCut: 232423|161779


In [55]:
Plots.PlotTagged(samples["mc"].recoParticles.beam_endPos.z, Tags.GenerateTrueBeamParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.beam_endPos.z, x_label = "Beam end position z (cm)", norm = args.norm, bins = 50, y_scale = "linear", ncols = 2, loc = "upper right")
Plots.Save("linear_stacked")
Plots.PlotTagged(samples["mc"].recoParticles.beam_endPos.z, Tags.GenerateTrueBeamParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.beam_endPos.z, x_label = "Beam end position z (cm)", norm = args.norm, bins = 50, y_scale = "log", stacked = False, ncols = 2, loc = "lower left", alpha = 0.7)
Plots.Save("log_unstacked")

ranges = [0, 222, 460, 700]
loc = ["lower left", "best", "best"]
for i in range(len(ranges)-1):
    Plots.PlotTagged(samples["mc"].recoParticles.beam_endPos.z, Tags.GenerateTrueBeamParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.beam_endPos.z, x_label = "Beam end position z (cm)", norm = args.norm, x_range = [ranges[i], ranges[i + 1]], title = f"{[ranges[i], ranges[i + 1]]} cm", bins = 50, y_scale = "log", ncols = 3, loc = loc[i], stacked = False, alpha = 0.7)
    Plots.Save(f"log_unstacked_{i}")
